## Import Dependencies

In [1]:
%%time

import sys
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from scipy.stats import qmc
import itertools
from itertools import combinations_with_replacement, combinations, permutations

import bo_methods_lib
from bo_methods_lib.bo_functions_generic import gen_theta_set, clean_1D_arrays
from bo_methods_lib.GPBO_Classes_New import * #Fix this later
from bo_methods_lib.GPBO_Class_fxns import * #Fix this later

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CPU times: user 2.23 s, sys: 302 ms, total: 2.53 s
Wall time: 2.61 s


## Define Method and DateTime

In [2]:
#Generate Method (except last part)
meth_name = Method_name_enum(1)
method = GPBO_Methods(meth_name)

print("Method Name: ", method.method_name.name)
print("Emulator?: ", method.emulator)
print("Obj Func: ", method.obj.name)
print("Sparse Grid?: ", method.sparse_grid)

#Set Date and Time
dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%d-%b-%Y (%H:%M:%S)")
# print("Date and Time: ", timestampStr)
# DateTime = dateTimeObj.strftime("%Y/%m/%d/%H-%M-%S%p")
DateTime = dateTimeObj.strftime("%Y/%m/%d/%H-%M")
DateTime = None ##For Testing

Method Name:  A1
Emulator?:  False
Obj Func:  OBJ
Sparse Grid?:  False


## Define Case Study Simulator & Functions

In [3]:
def simulator_helper_test_fxns(cs_name, indecies_to_consider, noise_mean, noise_std, case_study_parameters):
    """
    Sets the model for calculating y based off of the case study identifier.

    Parameters
    ----------
    cs_name: Class, The name/enumerator associated with the case study being evaluated

    Returns
    -------
    calc_y_fxn: function, the function used for calculation is case study cs_name.name
    """
    #Note: Add your function name from GPBO_Class_fxns.py here
    if cs_name.value == 1:      
        theta_ref = np.array([1.0, -1.0])
        theta_names = ['theta_1', 'theta_2']
        bounds_x_l = [-2]
        bounds_x_u = [2]
        bounds_theta_l = [-2, -2]
        bounds_theta_u = [ 2,  2]
        calc_y_fxn = calc_cs1_polynomial
        
    elif cs_name.value == 2:     
        theta_ref = np.array([-200,-100,-170,15,-1,-1,-6.5,0.7,0,0,11,0.6,-10,-10,-6.5,0.7,1,0,-0.5,-1,0,0.5,1.5,1])
                             
        theta_names = ['A_1', 'A_2', 'A_3', 'A_4', 'a_1', 'a_2', 'a_3', 'a_4', 'b_1', 'b_2', 'b_3', 'b_4', 'c_1', 
                       'c_2', 'c_3', 'c_4', 'x0_1', 'x0_2', 'x0_3', 'x0_4', 'x1_1', 'x1_2', 'x1_3', 'x1_4']
        bounds_x_l = [-1.5, -0.5]
        bounds_x_u = [1, 2]
        bounds_theta_l = [-300,-200,-250, 5,-2,-2,-10, -2, -2,-2,5,-2,-20,-20, -10,-1 ,-2,-2,-2, -2,-2,-2,0,-2]
        bounds_theta_u = [-100,  0, -150, 20,2, 2, 0,  2,  2,  2, 15,2, 0,0   , 0,  2, 2,  2, 2, 2 ,2 , 2, 2,2]
        calc_y_fxn = calc_muller
        
    else:
        raise ValueError("self.CaseStudyParameters.cs_name.value must exist!")

    return Simulator(indecies_to_consider, 
                     theta_ref,
                     theta_names,
                     bounds_theta_l, 
                     bounds_x_l, 
                     bounds_theta_u, 
                     bounds_x_u, 
                     noise_mean,
                     noise_std,
                     case_study_parameters,
                     calc_y_fxn)

## Define Case Study

In [4]:
cs_name  = CS_name_enum(1)
print("CS Name: ", cs_name.name)

# indecies_to_consider = list(range(4, 12)) #This is what changes for different subproblems of CS2
indecies_to_consider = list(range(0, 2)) #This is what changes for different subproblems of CS1

ep0 = 1
sep_fact = 0.8
normalize = False
lhs_gen_theta = True
eval_all_pairs = False
noise_mean = 0
noise_std = 0.01
noise_std = 0
kernel = Kernel_enum(1)
set_lenscl = 1
outputscl = False
retrain_GP = 2
GP_train_iter = 300
bo_iter_tot = 3
bo_run_tot = 2
save_fig = False
save_data = False
num_data = None
seed = 1

cs_params = CaseStudyParameters(cs_name, ep0, sep_fact, normalize, eval_all_pairs, bo_iter_tot, bo_run_tot, 
                         save_fig, save_data, DateTime, seed)

simulator = simulator_helper_test_fxns(cs_name, indecies_to_consider, noise_mean, noise_std, cs_params)

CS Name:  CS1


## Generate Experimental X and Y Data (If not given)

In [5]:
#Test gen_exp_data
num_x_data = 5
gen_meth_x = Gen_meth_enum(2)
# num_x_data = 27
# gen_meth_x = Gen_meth_enum(1)

exp_data = simulator.gen_exp_data(num_x_data, gen_meth_x)
print(len(exp_data.theta_vals) == len(exp_data.x_vals) == len(exp_data.y_vals))
# print(exp_data.theta_vals)
# print(exp_data.x_vals)

True


## Generate Simulation Theta and y Data

In [20]:
num_theta_data = 20
gen_meth_theta = Gen_meth_enum(1)
sim_data = simulator.gen_sim_data(method, num_theta_data, num_x_data, gen_meth_theta, gen_meth_x)
sim_sse_data = simulator.sim_data_to_sse_sim_data(method, sim_data, exp_data)

## Create Normalized Data (optional)

In [22]:
#What is the best way to be building normalized data? Is it ok to write a method in a class that calls itself?
scaled_exp_data = exp_data.norm_feature_data(simulator)
unscaled_exp_data =  scaled_exp_data.unnorm_feature_data(simulator)

#How do I best automate which one of these to use? Where would I automate this?
scaled_sim_data = sim_data.norm_feature_data(simulator)
unscaled_sim_data =  scaled_sim_data.unnorm_feature_data(simulator)

scaled_sim_sse_data = sim_sse_data.norm_feature_data(simulator)
unscaled_sim_sse_data =  scaled_sim_sse_data.unnorm_feature_data(simulator)
# print(unscaled_exp_data.theta_vals)
# print(unscaled_exp_data.x_vals)

## Generate, Shuffle, and Split Type 1 Training Data

In [23]:
#Need this to get training indecies based on emulator status
#Should it call method class? SHould I move it to GP class?
train_idx, test_idx = sim_sse_data.train_test_idx_split(cs_params)
# train_data, test_data = driver.get_train_test_data(sim_data, train_idx, test_idx)
print(len(test_idx), test_idx)

1
4 [ 8  2 18  4]


## Build GP Model

In [9]:
gp_emulator = GP_Emulator(CS, kernel, set_lenscl, outputscl, retrain_GP, GP_train_iter)

NameError: name 'CS' is not defined

## Augmenting Training Data (Skipped Steps)

In [ ]:
best_theta = Data(None, exp_data.x_vals, None, None, None, None, None)

#Note: Will generate this using a method in driver that doesn't exist yet but generates experimental data instead of sim data
if "2" in method.method_name.name:
    best_theta.theta_vals = np.repeat(simulator.theta_true.reshape(1,-1), exp_data.get_num_x_vals(), axis = 0)
    best_theta.y_vals = simulator.create_sim_data(CS, best_theta)
else:
    best_theta.theta_vals = simulator.theta_true.reshape(1,-1)
    best_theta.y_vals = simulator.create_sse_sim_data(method, CS, best_theta, exp_data)

print(len(sim_data.theta_vals), len(sim_data.y_vals))
sim_data = driver.augment_train_data(sim_data, best_theta) #Best error in driver class
print(len(sim_data.theta_vals), len(sim_data.y_vals))

# Case Study 1 Test

In [ ]:
#Write Later